In [1]:
from datasets import load_dataset

huggingface_dataset_name = "multi_news"
dataset = load_dataset(huggingface_dataset_name)
dataset

/Users/soli/Documents/Projects/llm/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 5622/5622 [00:00<00:00, 18314.74 examples/s]


DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [2]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [5]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [6]:
dataset["train"]

Dataset({
    features: ['document', 'summary'],
    num_rows: 44972
})

In [10]:
dataset["train"][0]["document"]

'National Archives \n \n Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. \n \n A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. \n \n Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you’re here, why don’t you sign up to follow us on Twitter. \n \n Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be gr

In [11]:
dataset["test"]

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [12]:
dataset["validation"]

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})